<a href="https://colab.research.google.com/github/maochaokuo/pyKerasGA1/blob/master/twfutures1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tw futures tendency prediction by keras
## place to imort


In [1]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing
from datetime import datetime
import datetime
import io
import sys
#sys.exit()
print(datetime.datetime.now())

#PERCENT=0.2 # normaly 0.067 for a whole year
SEQ_LEN = 20  # how long of a preceeding sequence to collect for RNN

2019-08-03 15:23:31.883838


## orignal data to process (predict the tendency)

In [2]:
url='https://raw.githubusercontent.com/maochaokuo/tensorflow1/master/sharedfiles/aOHLCday.csv'
df = pd.read_csv(url, names=['dealdate','dealtime','open','high', 'low', 'close', 'volume'])  # read in specific file

'''
url2='https://raw.githubusercontent.com/maochaokuo/tensorflow1/master/sharedfiles/aOHLC.csv'
dfall = pd.read_csv(url2, names=['dealdate','dealtime','open','high', 'low', 'close', 'volume'])  # read in specific file
'''
#cols=["dealdate","dealtime","open","high", "low", "close", "volume"]
#df = pd.read_csv(url, names=cols, dtype=dict(zip(cols,[str, str, float, float, float, float, int])) ) 
#df.info()

#df['dealdate']=df['dealdate'].astype('|S')

#df.dealdate = df.dealdate.astype(str)
#df.dealtime = df.dealtime.astype(str)
#df.open=df.open.astype(float)
#df['dealtime']='0'+df['dealtime']
#df['dealtime']=df['dealtime'][-6:]
df=df.iloc[1:] #remove 1st row, the header
#dfall=dfall.iloc[1:]

dropIndexes=df[ df['dealdate']>'20180731' ].index
df.drop(dropIndexes, inplace=True)

'''
dfmorning=dfall.copy()
dropIndexes=dfmorning[ dfmorning['dealtime']>'060000' ].index
dfmorning.drop(dropIndexes, inplace=True)

dfnight=dfall.copy()
dropIndexes=dfnight[ dfnight['dealtime']<'140000'].index
dfnight.drop(dropIndexes, inplace=True)
'''
#df.groupby('dealdate')['open'].first()
#df.groupby('dealdate')['close'].last()
#df.groupby('dealdate')['low'].min()
#df.groupby('dealdate')['high'].max()
print(datetime.datetime.now()) # about 0.5 second
#df['dealdate'].dtype


2019-08-03 15:23:32.528738


In [0]:

df['open'] = pd.to_numeric(df['open'], downcast='float')
df['high'] = pd.to_numeric(df['high'], downcast='float')
df['low'] = pd.to_numeric(df['low'], downcast='float')
df['close'] = pd.to_numeric(df['close'], downcast='float')
df['volume'] = pd.to_numeric(df['volume'], downcast='float')


In [4]:
#dfNext['dealdate']=str(dfNext['dealdate'])
dfNext=df
for index, row in dfNext.iterrows():
    dealdate=row['dealdate']#[:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]
    dfNext.at[index,'dealdate']=datetime.datetime.strptime(dealdate, '%Y%m%d')
    dfNext.at[index,'dealtime']=datetime.datetime.strptime(row['dealtime'], '%H%M%S')
    #dfNext.at[index,'dealtime']=datetime.datetime.strptime(dealdate[:4]+'-'+dealdate[4:6]+'-'+dealdate[6:8])
    #dfNext.at[index,'dealdate']=row['dealdate'][:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]
    #dfNext.at[index,'dealtime']=row['dealtime'][:2]+':'+row['dealtime'][2:4]+':'+row['dealtime'][4:]
    dfNext.at[index,'dealdatetime']=row['dealdate'][:4]+'-'+row['dealdate'][4:6]+'-'+row['dealdate'][6:]+' '+row['dealtime'][:2]+':'+row['dealtime'][2:4]+':'+row['dealtime'][4:]

dfNext['dealdatetime']= pd.to_datetime(dfNext['dealdatetime'])
dfNext['dealtime']=dfNext['dealdatetime'].dt.time
dfNext['dealdate']=dfNext['dealdatetime'].dt.date
#dfNext['dealtime']= pd.to_datetime(dfNext['dealdatetime']).time()
print(dfNext.head())
print(dfNext.dtypes)
#wrong!!!!!!!!!!!!!!!!!!

     dealdate  dealtime     open  ...    close  volume        dealdatetime
1  2018-06-29  08:45:00  10541.0  ...  10531.0  4426.0 2018-06-29 08:45:00
2  2018-06-29  08:46:00  10531.0  ...  10531.0  2142.0 2018-06-29 08:46:00
3  2018-06-29  08:47:00  10530.0  ...  10532.0   454.0 2018-06-29 08:47:00
4  2018-06-29  08:48:00  10531.0  ...  10536.0  1204.0 2018-06-29 08:48:00
5  2018-06-29  08:49:00  10537.0  ...  10527.0  1522.0 2018-06-29 08:49:00

[5 rows x 8 columns]
dealdate                object
dealtime                object
open                   float32
high                   float32
low                    float32
close                  float32
volume                 float32
dealdatetime    datetime64[ns]
dtype: object


In [5]:

for index, row in dfNext.iterrows():
    dfNext.at[index,'dealdateI']=int(row['dealdatetime'].strftime("%Y%m%d"))
    dfNext.at[index,'dealtimeI']=int(row['dealdatetime'].strftime("%H%M%S"))
dfNext.head()

,dealdate,dealtime,open,high,low,close,volume,dealdatetime,dealdateI,dealtimeI
1,2018-06-29,08:45:00,10541.0,10542.0,10529.0,10531.0,4426.0,2018-06-29 08:45:00,20180629.0,84500.0
2,2018-06-29,08:46:00,10531.0,10532.0,10526.0,10531.0,2142.0,2018-06-29 08:46:00,20180629.0,84600.0
3,2018-06-29,08:47:00,10530.0,10532.0,10530.0,10532.0,454.0,2018-06-29 08:47:00,20180629.0,84700.0
4,2018-06-29,08:48:00,10531.0,10538.0,10529.0,10536.0,1204.0,2018-06-29 08:48:00,20180629.0,84800.0
5,2018-06-29,08:49:00,10537.0,10537.0,10527.0,10527.0,1522.0,2018-06-29 08:49:00,20180629.0,84900.0


In [6]:

df.dtypes
#df1=df(['dealdate','dealtime','open','high','low','close','volume'], dtypes=[np.str,np.str,np.float64,np.float64,np.float64,np.float64,np.int64])
#df.dtypes
df.head()

,dealdate,dealtime,open,high,low,close,volume,dealdatetime,dealdateI,dealtimeI
1,2018-06-29,08:45:00,10541.0,10542.0,10529.0,10531.0,4426.0,2018-06-29 08:45:00,20180629.0,84500.0
2,2018-06-29,08:46:00,10531.0,10532.0,10526.0,10531.0,2142.0,2018-06-29 08:46:00,20180629.0,84600.0
3,2018-06-29,08:47:00,10530.0,10532.0,10530.0,10532.0,454.0,2018-06-29 08:47:00,20180629.0,84700.0
4,2018-06-29,08:48:00,10531.0,10538.0,10529.0,10536.0,1204.0,2018-06-29 08:48:00,20180629.0,84800.0
5,2018-06-29,08:49:00,10537.0,10537.0,10527.0,10527.0,1522.0,2018-06-29 08:49:00,20180629.0,84900.0


## df2 whole day summary (#1)

In [7]:
df2=pd.DataFrame( df.groupby('dealdate')['open'].first() )
df2=df2.join(df.groupby('dealdate')['high'].max())
df2=df2.join(df.groupby('dealdate')['low'].min())
df2=df2.join(df.groupby('dealdate')['close'].last())
df2=df2.join(df.groupby('dealdate')['dealdate'].last())
#df2['dealdate']=df2['name']
df2['hightime']='140000'
df2['lowtime']='140000'
l=1
for index, row in df2.iterrows():
    df2.at[index,'i']=int(l)
    l=l+1
lastclose=0
for index, row in df2.iterrows():
    if lastclose==0:
        lastclose=row['close']
        if row['dealdate']==datetime.date(2018,6,29):
            df2.at[index,'lastclose']=10523
        continue
    df2.at[index,'lastclose']=lastclose
    lastclose=row['close']
#df2[df2['i']]
print(datetime.datetime.now()) # about 0.2 second
df2.head()


2019-08-03 15:23:35.927992


,open,high,low,close,dealdate,hightime,lowtime,i,lastclose
dealdate,,,,,,,,,
2018-06-29,10541.0,10692.0,10526.0,10677.0,2018-06-29,140000,140000,1.0,10523.0
2018-07-02,10680.0,10719.0,10609.0,10611.0,2018-07-02,140000,140000,2.0,10677.0
2018-07-03,10651.0,10717.0,10544.0,10622.0,2018-07-03,140000,140000,3.0,10611.0
2018-07-04,10598.0,10639.0,10584.0,10618.0,2018-07-04,140000,140000,4.0,10622.0
2018-07-05,10625.0,10633.0,10488.0,10500.0,2018-07-05,140000,140000,5.0,10618.0


## df2 whole day summary (#2)

In [8]:
for index, row in df2.iterrows():
    df3=df[(df['dealdate']==row['dealdate']) & (df['high']==row['high'])] #first dealtime when high hit highest
    #row['hightime']=df3['dealtime'].iloc[0]
    df2.at[index,'hightime']=df3['dealtime'].iloc[0]
    df3=df[(df['dealdate']==row['dealdate']) & (df['low']==row['low'])] #first dealtime when low hit lowest
    df2.at[index,'lowtime']=df3['dealtime'].iloc[0]
    #row['hightime']=
    #df3.iloc[0]['hightime']  #df[['dealtime']][ (df['dealdate']==row['dealdate']) & (df['high']==row['high'])]
    
#df3= df[(df['dealdate']=='20180629') & (df['high']=='10692.00')] 
#df3.iloc[0]
print(datetime.datetime.now()) # about 5 seconds
df2.head()
#df2.tail()

2019-08-03 15:23:36.075469


,open,high,low,close,dealdate,hightime,lowtime,i,lastclose
dealdate,,,,,,,,,
2018-06-29,10541.0,10692.0,10526.0,10677.0,2018-06-29,13:29:00,08:46:00,1.0,10523.0
2018-07-02,10680.0,10719.0,10609.0,10611.0,2018-07-02,10:15:00,13:43:00,2.0,10677.0
2018-07-03,10651.0,10717.0,10544.0,10622.0,2018-07-03,09:25:00,11:21:00,3.0,10611.0
2018-07-04,10598.0,10639.0,10584.0,10618.0,2018-07-04,09:38:00,11:17:00,4.0,10622.0
2018-07-05,10625.0,10633.0,10488.0,10500.0,2018-07-05,08:45:00,13:13:00,5.0,10618.0


接下來算大趨勢

!!!>>>>> 目前這裡非常慢

In [9]:
print(df.dtypes)
print(df2.dtypes)

dealdate                object
dealtime                object
open                   float32
high                   float32
low                    float32
close                  float32
volume                 float32
dealdatetime    datetime64[ns]
dealdateI              float64
dealtimeI              float64
dtype: object
open         float32
high         float32
low          float32
close        float32
dealdate      object
hightime      object
lowtime       object
i            float64
lastclose    float64
dtype: object


In [10]:
#df.head(10)
#print(datetime.datetime.now())    # about 1:22 seconds, or 82 seconds
print(datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
daterow=None
curdate=datetime.datetime(2000, 1, 1).date() #'20000101'
l=0
for index, row in df.iterrows():
    dealdate=row['dealdate']
    dealtime=row['dealtime']
    # daily summary
    if daterow is None or curdate<dealdate:
        daterow=df2[ df2['dealdate']==dealdate ].iloc[0] 
        datehightime=daterow['hightime']
        datelowtime=daterow['lowtime']
    else:
        curdate=dealdate
    if datehightime < datelowtime: # high earlier than low
        if dealtime < datehightime:
            df.at[index,'tendMajor']=1
        elif dealtime >= datehightime and dealtime < datelowtime:
            df.at[index,'tendMajor']=-1
        else:
            df.at[index,'tendMajor']=1
    else: # low earlier than high
        if dealtime < datelowtime:
            df.at[index,'tendMajor']=-1
        elif dealtime >= datelowtime and dealtime < datehightime:
            df.at[index,'tendMajor']=1
        else:
            df.at[index,'tendMajor']=-1
    l=l+1
print('l='+str(l))
#print(datetime.datetime.now())    # about 1:22 seconds, or 82 seconds
print(datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
#df.head(300)

08/03/2019, 15:23:36
l=6890
08/03/2019, 15:23:44


In [11]:
df.head(10)

,dealdate,dealtime,open,high,low,close,volume,dealdatetime,dealdateI,dealtimeI,tendMajor
1,2018-06-29,08:45:00,10541.0,10542.0,10529.0,10531.0,4426.0,2018-06-29 08:45:00,20180629.0,84500.0,-1.0
2,2018-06-29,08:46:00,10531.0,10532.0,10526.0,10531.0,2142.0,2018-06-29 08:46:00,20180629.0,84600.0,1.0
3,2018-06-29,08:47:00,10530.0,10532.0,10530.0,10532.0,454.0,2018-06-29 08:47:00,20180629.0,84700.0,1.0
4,2018-06-29,08:48:00,10531.0,10538.0,10529.0,10536.0,1204.0,2018-06-29 08:48:00,20180629.0,84800.0,1.0
5,2018-06-29,08:49:00,10537.0,10537.0,10527.0,10527.0,1522.0,2018-06-29 08:49:00,20180629.0,84900.0,1.0
6,2018-06-29,08:50:00,10527.0,10532.0,10526.0,10531.0,1308.0,2018-06-29 08:50:00,20180629.0,85000.0,1.0
7,2018-06-29,08:51:00,10531.0,10535.0,10529.0,10532.0,888.0,2018-06-29 08:51:00,20180629.0,85100.0,1.0
8,2018-06-29,08:52:00,10533.0,10534.0,10530.0,10530.0,506.0,2018-06-29 08:52:00,20180629.0,85200.0,1.0
9,2018-06-29,08:53:00,10530.0,10531.0,10528.0,10528.0,668.0,2018-06-29 08:53:00,20180629.0,85300.0,1.0
10,2018-06-29,08:54:00,10528.0,10533.0,10528.0,10533.0,608.0,2018-06-29 08:54:00,20180629.0,85400.0,1.0


接下來算小趨勢

In [12]:
curdate=datetime.datetime(2000, 1, 1).date() #'20000101'
pre2index=-1
preindex=-1
pre2high=-1.0
pre2low=-1.0
prehigh=-1.0
prelow=-1.0
l=0
df['tendMinor']=0
for index, row in df.iterrows():
    if row['dealdate']=='dealdate':
        continue
    if row['dealdate'] > curdate: ## change date
        pre2index = -1
        preindex = -1 
        pre2high=-1.0
        pre2low=-1.0
        prehigh=-1.0
        prelow=-1.0
    curdate=row['dealdate']
    tendMinor = 0
    #print(pre2high)
    #print(pre2low)
    #print(prehigh)
    #print(prelow)
    if pre2high>0 and pre2low>0 and prehigh>0 and prelow>0:
        if prehigh > pre2high and prehigh >float(row['high']): #local high at prerow
            tendMinor = -1
        if prelow < pre2low and prelow <float( row['low']): #local low at prerow
            if tendMinor!=0:
                tendMinor = 0
            else:
                tendMinor = 1
    #print('1 preindex='+str(preindex))
    if preindex>0:
        #print('prerow not none')
        df.at[preindex,'tendMinor']=tendMinor
    #else:
    #  print('prerow none')
    # before end of for
    pre2index = preindex
    preindex = index
    #print('2 preindex='+str(preindex))
    pre2high=prehigh
    pre2low=prelow
    try:
        prehigh=float(row['high'])
        prelow=float(row['low'])
    except:
        print(row)
        break
    l=l+1
    #if l>300:
    #  break
print('l='+str(l))
print(datetime.datetime.now())  # about 14 seconds
df.head(10)

l=6890
2019-08-03 15:23:45.945979


,dealdate,dealtime,open,high,low,close,volume,dealdatetime,dealdateI,dealtimeI,tendMajor,tendMinor
1,2018-06-29,08:45:00,10541.0,10542.0,10529.0,10531.0,4426.0,2018-06-29 08:45:00,20180629.0,84500.0,-1.0,0
2,2018-06-29,08:46:00,10531.0,10532.0,10526.0,10531.0,2142.0,2018-06-29 08:46:00,20180629.0,84600.0,1.0,1
3,2018-06-29,08:47:00,10530.0,10532.0,10530.0,10532.0,454.0,2018-06-29 08:47:00,20180629.0,84700.0,1.0,0
4,2018-06-29,08:48:00,10531.0,10538.0,10529.0,10536.0,1204.0,2018-06-29 08:48:00,20180629.0,84800.0,1.0,-1
5,2018-06-29,08:49:00,10537.0,10537.0,10527.0,10527.0,1522.0,2018-06-29 08:49:00,20180629.0,84900.0,1.0,0
6,2018-06-29,08:50:00,10527.0,10532.0,10526.0,10531.0,1308.0,2018-06-29 08:50:00,20180629.0,85000.0,1.0,1
7,2018-06-29,08:51:00,10531.0,10535.0,10529.0,10532.0,888.0,2018-06-29 08:51:00,20180629.0,85100.0,1.0,-1
8,2018-06-29,08:52:00,10533.0,10534.0,10530.0,10530.0,506.0,2018-06-29 08:52:00,20180629.0,85200.0,1.0,0
9,2018-06-29,08:53:00,10530.0,10531.0,10528.0,10528.0,668.0,2018-06-29 08:53:00,20180629.0,85300.0,1.0,0
10,2018-06-29,08:54:00,10528.0,10533.0,10528.0,10533.0,608.0,2018-06-29 08:54:00,20180629.0,85400.0,1.0,0


這裡試算損益以決定最佳趨勢

In [13]:
curdate=datetime.datetime(2000, 1, 1).date() #'20000101'
pretend=0
baseprice=0
inventoryPLmin=0
settlePLmin=0
settlePLsum=0
for index, row in df.iterrows():
    if row['dealdate']>curdate:
        curdate=row['dealdate']
        pretend=0
        baseprice=0
    dealtime=row['dealtime']
    close=float(row['close'])
    earliest=datetime.datetime.now().replace(hour=9, minute=5, second=0, microsecond=0).time()
    latest=datetime.datetime.now().replace(hour=13, minute=35, second=0, microsecond=0).time()
    #print(earliest)
    #print(latest)
    if row['dealtime'] < earliest or row['dealtime'] > latest :
        curtend=0
    else:
        tendMajor=int(df.at[index, 'tendMajor'])
        if np.isnan( df.at[index, 'tendMinor'] ):
            print('wrong! '+curdate+' '+dealtime)
        tendMinor=int(df.at[index, 'tendMinor'])
        tendMinor=int(df.at[index, 'tendMinor'])
        if tendMinor!=0:
            curtend=tendMinor
        else:
            #curtend=pretend
            if pretend!=0:
                curtend=pretend
            else:
                curtend=tendMajor
    inventoryPL=0
    settlePL=0
    if curtend!=pretend:
        if pretend!=0 and baseprice==0:
            print('wrong! '+curdate+' '+dealtime)
            break
        settlePL=pretend*(close-baseprice)
        if settlePL<settlePLmin:
            settlePLmin = settlePL
        settlePLsum=settlePLsum+settlePL
        if curtend==0:
            baseprice=0
        else:
            baseprice=close
    else: #curtend==pretend
        inventoryPL=curtend*(close-baseprice)
        if inventoryPL<inventoryPLmin:
            inventoryPLmin = inventoryPL
    df.at[index, 'testTend']=curtend
    df.at[index, 'baseprice']=baseprice
    df.at[index, 'inventoryPL']=inventoryPL
    df.at[index, 'inventoryPLmin']=inventoryPLmin
    df.at[index, 'settlePL']=settlePL
    df.at[index, 'settlePLmin']=settlePLmin
    df.at[index, 'settlePLsum']=settlePLsum
    pretend=curtend
print(df.drop(["open","high","low","volume"], axis=1).tail(10))
#df[ df['dealdate']=='20180702']    
print(datetime.datetime.now())  # about 14 seconds

        dealdate  dealtime    close  ... settlePL  settlePLmin  settlePLsum
6881  2018-07-31  13:35:00  10990.0  ...      0.0        -41.0       3311.0
6882  2018-07-31  13:36:00  10991.0  ...     14.0        -41.0       3325.0
6883  2018-07-31  13:37:00  10993.0  ...      0.0        -41.0       3325.0
6884  2018-07-31  13:38:00  10993.0  ...      0.0        -41.0       3325.0
6885  2018-07-31  13:39:00  10991.0  ...      0.0        -41.0       3325.0
6886  2018-07-31  13:40:00  10991.0  ...      0.0        -41.0       3325.0
6887  2018-07-31  13:41:00  10991.0  ...      0.0        -41.0       3325.0
6888  2018-07-31  13:42:00  10992.0  ...      0.0        -41.0       3325.0
6889  2018-07-31  13:43:00  10992.0  ...      0.0        -41.0       3325.0
6890  2018-07-31  13:44:00  10989.0  ...      0.0        -41.0       3325.0

[10 rows x 15 columns]
2019-08-03 15:23:48.184508


In [14]:
df2.head(10)

,open,high,low,close,dealdate,hightime,lowtime,i,lastclose
dealdate,,,,,,,,,
2018-06-29,10541.0,10692.0,10526.0,10677.0,2018-06-29,13:29:00,08:46:00,1.0,10523.0
2018-07-02,10680.0,10719.0,10609.0,10611.0,2018-07-02,10:15:00,13:43:00,2.0,10677.0
2018-07-03,10651.0,10717.0,10544.0,10622.0,2018-07-03,09:25:00,11:21:00,3.0,10611.0
2018-07-04,10598.0,10639.0,10584.0,10618.0,2018-07-04,09:38:00,11:17:00,4.0,10622.0
2018-07-05,10625.0,10633.0,10488.0,10500.0,2018-07-05,08:45:00,13:13:00,5.0,10618.0
2018-07-06,10535.0,10556.0,10414.0,10539.0,2018-07-06,09:00:00,11:10:00,6.0,10500.0
2018-07-09,10583.0,10676.0,10582.0,10661.0,2018-07-09,11:38:00,08:45:00,7.0,10539.0
2018-07-10,10733.0,10744.0,10680.0,10709.0,2018-07-10,08:46:00,11:01:00,8.0,10661.0
2018-07-11,10608.0,10656.0,10574.0,10617.0,2018-07-11,10:45:00,09:35:00,9.0,10709.0


In [15]:

df.drop(["open","high","low","volume"], axis=1).tail(10)

,dealdate,dealtime,close,dealdatetime,dealdateI,dealtimeI,tendMajor,tendMinor,testTend,baseprice,inventoryPL,inventoryPLmin,settlePL,settlePLmin,settlePLsum
6881,2018-07-31,13:35:00,10990.0,2018-07-31 13:35:00,20180731.0,133500.0,-1.0,0,1.0,10977.0,13.0,-47.0,0.0,-41.0,3311.0
6882,2018-07-31,13:36:00,10991.0,2018-07-31 13:36:00,20180731.0,133600.0,-1.0,0,0.0,0.0,0.0,-47.0,14.0,-41.0,3325.0
6883,2018-07-31,13:37:00,10993.0,2018-07-31 13:37:00,20180731.0,133700.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0
6884,2018-07-31,13:38:00,10993.0,2018-07-31 13:38:00,20180731.0,133800.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0
6885,2018-07-31,13:39:00,10991.0,2018-07-31 13:39:00,20180731.0,133900.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0
6886,2018-07-31,13:40:00,10991.0,2018-07-31 13:40:00,20180731.0,134000.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0
6887,2018-07-31,13:41:00,10991.0,2018-07-31 13:41:00,20180731.0,134100.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0
6888,2018-07-31,13:42:00,10992.0,2018-07-31 13:42:00,20180731.0,134200.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0
6889,2018-07-31,13:43:00,10992.0,2018-07-31 13:43:00,20180731.0,134300.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0
6890,2018-07-31,13:44:00,10989.0,2018-07-31 13:44:00,20180731.0,134400.0,-1.0,0,0.0,0.0,0.0,-47.0,0.0,-41.0,3325.0


In [16]:

print(datetime.datetime.now())  
dfNext=df.drop(["tendMajor","tendMinor","baseprice","inventoryPL","inventoryPLmin","settlePL","settlePLmin","settlePLsum"], axis=1)

for index, row in dfNext.iterrows():
    #dfNext.at[index, 'lastclose']=df2[ df2["dealdate"]==row["dealdate"] ]["lastclose"]
    df3=df2[ df2['dealdate']==row['dealdate'] ] #first dealtime when low hit lowest
    dfNext.at[index,'lastclose']=df3['lastclose'].iloc[0]
print(dfNext.head())
print(dfNext.tail())
print(datetime.datetime.now())  

2019-08-03 15:23:48.305287
     dealdate  dealtime     open  ...  dealtimeI  testTend  lastclose
1  2018-06-29  08:45:00  10541.0  ...    84500.0       0.0    10523.0
2  2018-06-29  08:46:00  10531.0  ...    84600.0       0.0    10523.0
3  2018-06-29  08:47:00  10530.0  ...    84700.0       0.0    10523.0
4  2018-06-29  08:48:00  10531.0  ...    84800.0       0.0    10523.0
5  2018-06-29  08:49:00  10537.0  ...    84900.0       0.0    10523.0

[5 rows x 12 columns]
        dealdate  dealtime     open  ...  dealtimeI  testTend  lastclose
6886  2018-07-31  13:40:00  10990.0  ...   134000.0       0.0    10940.0
6887  2018-07-31  13:41:00  10990.0  ...   134100.0       0.0    10940.0
6888  2018-07-31  13:42:00  10992.0  ...   134200.0       0.0    10940.0
6889  2018-07-31  13:43:00  10991.0  ...   134300.0       0.0    10940.0
6890  2018-07-31  13:44:00  10991.0  ...   134400.0       0.0    10940.0

[5 rows x 12 columns]
2019-08-03 15:23:55.146714


接下來應該要用到的欄位：

1. dealdate
2. dealtime
3. lastclose (前一交易日日盤)
4. X preclose (夜盤收盤, 目前有困難)
5. open
6. high
7. low
8. close
9. volume
10. rsi ? if any



In [17]:

print(datetime.datetime.now())  
dfNext['date'] = pd.to_datetime(dfNext['dealdatetime'])
dfNext['close'] = pd.to_numeric(dfNext['close'], downcast='float')
dfNext.dtypes


2019-08-03 15:23:55.159030


dealdate                object
dealtime                object
open                   float32
high                   float32
low                    float32
close                  float32
volume                 float32
dealdatetime    datetime64[ns]
dealdateI              float64
dealtimeI              float64
testTend               float64
lastclose              float64
date            datetime64[ns]
dtype: object

In [18]:
!pip install stockstats
import stockstats
from stockstats import StockDataFrame as Sdf

stock = Sdf.retype(dfNext)

print(datetime.datetime.now())  

2019-08-03 15:24:00.515740


In [19]:

dfNext['macd']=stock['macd']
dfNext['rsi_6']=stock['rsi_6']
dfNext['rsi_12']=stock['rsi_12']
dfNext['rsi_14']=stock['rsi_14']
dfNext['kdjk']=stock['kdjk']
dfNext['kdjd']=stock['kdjd']
dfNext['kdjj']=stock['kdjj']
dfNext['boll']=stock['boll']
dfNext['boll_ub']=stock['boll_ub']
dfNext['boll_lb']=stock['boll_lb']
dfNext['wr_10']=stock['wr_10']
dfNext['wr_6']=stock['wr_6']
dfNext['dma']=stock['dma']
dfNext['pdi']=stock['pdi']
dfNext['mdi']=stock['mdi']
dfNext['dx']=stock['dx']
dfNext['adx']=stock['adx']
dfNext['adxr']=stock['adxr']
dfNext['vr']=stock['vr']
dfNext['vr_6_sma']=stock['vr_6_sma']

#dfNext['close_-1_d']=dfNext['lastclose']
#dfNext["rsi_6"]=stock["rsi_6"]
#dfNext=dfNext.drop(["close_12_ema","close_26_ema","close_-1_s"], axis=1)
#dfNext=dfNext.drop(["close_-1_d"], axis=1)
print(datetime.datetime.now())  
dfNext.to_csv('dfNext.csv', mode='a', header=True, index=False)
df2.to_csv('df2.csv', mode='a', header=True, index=False)
print(dfNext.dtypes)
dfNext.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


2019-08-03 15:24:00.977056
dealdate                object
dealtime                object
open                   float32
high                   float32
low                    float32
close                  float32
volume                 float32
dealdatetime    datetime64[ns]
dealdatei              float64
dealtimei              float64
testtend               float64
lastclose              float64
close_12_ema           float64
close_26_ema           float64
macd                   float64
macds                  float64
macdh                  float64
close_-1_s             float32
close_-1_d             float32
rs_6                   float64
rsi_6                  float64
rs_12                  float64
rsi_12                 float64
rs_14                  float64
rsi_14                 float64
rsv_9                  float64
kdjk_9                 float64
kdjk                   float64
kdjd_9                 float64
kdjd                   float64
                     ...      
wr_10       

,dealdate,dealtime,open,high,low,close,volume,dealdatetime,dealdatei,dealtimei,testtend,lastclose,close_12_ema,close_26_ema,macd,macds,macdh,close_-1_s,close_-1_d,rs_6,rsi_6,rs_12,rsi_12,rs_14,rsi_14,rsv_9,kdjk_9,kdjk,kdjd_9,kdjd,kdjj_9,kdjj,close_20_sma,close_20_mstd,boll,boll_ub,boll_lb,wr_10,wr_6,close_10_sma,close_50_sma,dma,high_delta,um,low_delta,dm,pdm,pdm_14_ema,pdm_14,tr,atr_14,pdi_14,pdi,mdm,mdm_14_ema,mdm_14,mdi_14,mdi,dx_14,dx,dx_6_ema,adx,adx_6_ema,adxr,change,vr,vr_6_sma
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-06-29 08:45:00,2018-06-29,08:45:00,10541.0,10542.0,10529.0,10531.0,4426.0,2018-06-29 08:45:00,20180629.0,84500.0,0.0,10523.0,10531.000000,10531.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.384615,38.461538,38.461538,46.153846,46.153846,23.076923,23.076923,10531.000000,NaN,10531.000000,NaN,NaN,84.615385,84.615385,10531.000000,10531.000000,0.0,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-29 08:46:00,2018-06-29,08:46:00,10531.0,10532.0,10526.0,10531.0,2142.0,2018-06-29 08:46:00,20180629.0,84600.0,0.0,10523.0,10531.000000,10531.000000,0.000000,0.000000,0.000000,10531.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,31.250000,36.057692,36.057692,42.788462,42.788462,22.596154,22.596154,10531.000000,0.000000,10531.000000,10531.000000,10531.000000,68.750000,68.750000,10531.000000,10531.000000,0.0,-10.0,0.0,-3.0,3.0,0.0,0.000000,0.000000,6.0,6.000000,0.000000,0.000000,3.0,1.607143,1.607143,26.785714,26.785714,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,100.000000,100.000000
2018-06-29 08:47:00,2018-06-29,08:47:00,10530.0,10532.0,10530.0,10532.0,454.0,2018-06-29 08:47:00,20180629.0,84700.0,0.0,10523.0,10531.390300,10531.359290,0.031010,0.012709,0.036602,10531.0,1.0,inf,100.000000,inf,100.000000,inf,100.000000,37.500000,36.538462,36.538462,40.705128,40.705128,28.205128,28.205128,10531.333333,0.577350,10531.333333,10532.488034,10530.178633,62.500000,62.500000,10531.333333,10531.333333,0.0,0.0,0.0,4.0,0.0,0.0,0.000000,0.000000,2.0,3.925926,0.000000,0.000000,0.0,0.993209,0.993209,25.298715,25.298715,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.009501,142.390289,121.195145
2018-06-29 08:48:00,2018-06-29,08:48:00,10531.0,10538.0,10529.0,10536.0,1204.0,2018-06-29 08:48:00,20180629.0,84800.0,0.0,10523.0,10532.845402,10532.656630,0.188772,0.072351,0.232843,10532.0,4.0,inf,100.000000,inf,100.000000,inf,100.000000,62.500000,45.192308,45.192308,42.200855,42.200855,51.175214,51.175214,10532.500000,2.380476,10532.500000,10537.260952,10527.739048,37.500000,37.500000,10532.500000,10532.500000,0.0,6.0,6.0,-1.0,1.0,6.0,1.835569,1.835569,9.0,5.744059,31.955964,31.955964,0.0,0.689358,0.689358,12.001240,12.001240,45.395800,45.395800,75.453158,75.453158,88.965181,88.965181,0.037980,254.808590,165.732960
2018-06-29 08:49:00,2018-06-29,08:49:00,10537.0,10537.0,10527.0,10527.0,1522.0,2018-06-29 08:49:00,20180629.0,84900.0,0.0,10523.0,10531.257227,10531.344834,-0.087607,0.024767,-0.224748,10536.0,-9.0,0.447531,30.916844,0.500772,33.367609,0.508503,33.709132,6.250000,32.211538,32.211538,38.871083,38.871083,18.892450,18.892450,10531.400000,3.209361,10531.400000,10537.818723,10524.981277,93.750000,93.750000,10531.400000,10531.400000,0.0,-1.0,0.0,-2.0,2.0,0.0,1.356672,1.356672,10.0,6.929072,19.579420,19.579420,2.0,1.031303,1.031303,14.883706,14.883706,13.625328,13.625328,51.571462,51.571462,74.521436,74.521436,-0.085419,105.244890,150.610942


# 可以從這裡開始重複

In [20]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing
from datetime import datetime
import datetime
import io
import sys
#sys.exit()
print(datetime.datetime.now())

#PERCENT=0.2 # normaly 0.067 for a whole year
SEQ_LEN = 20  # how long of a preceeding sequence to collect for RNN

2019-08-03 15:24:01.900660


In [21]:
#may continue from here without rerun

#這邊搞定nan與infinite
dfNext=pd.read_csv('dfNext.csv')
df2=pd.read_csv('df2.csv')

dfNext=dfNext.replace([np.inf, -np.inf], np.nan) # replace all inf (infinite) with nan
dfNext['close_-1_s'].fillna(dfNext['close'], inplace=True)
dfNext['close_-1_d'].fillna(0, inplace=True)
dfNext['rs_6'].fillna(0, inplace=True)
dfNext['rsi_6'].fillna(100, inplace=True)
dfNext['rs_12'].fillna(0, inplace=True)
dfNext['rsi_12'].fillna(100, inplace=True)
dfNext['rs_14'].fillna(0, inplace=True)
dfNext['rsi_14'].fillna(100, inplace=True)
dfNext['close_20_mstd'].fillna(0, inplace=True)
dfNext['boll_ub'].fillna(dfNext['close'], inplace=True)
dfNext['boll_lb'].fillna(dfNext['close'], inplace=True)
dfNext['high_delta'].fillna(0, inplace=True)
dfNext['um'].fillna(0, inplace=True)
dfNext['low_delta'].fillna(0, inplace=True)
dfNext['dm'].fillna(0, inplace=True)
dfNext['tr'].fillna(0, inplace=True)
dfNext['atr_14'].fillna(0, inplace=True)
dfNext['pdi_14'].fillna(0, inplace=True)
dfNext['pdi'].fillna(0, inplace=True)
dfNext['mdi_14'].fillna(0, inplace=True)
dfNext['mdi'].fillna(0, inplace=True)
dfNext['dx_14'].fillna(100, inplace=True)
dfNext['dx'].fillna(100, inplace=True)
dfNext['dx_6_ema'].fillna(100, inplace=True)
dfNext['adx'].fillna(100, inplace=True)
dfNext['adx_6_ema'].fillna(100, inplace=True)
dfNext['adxr'].fillna(100, inplace=True)
dfNext['change'].fillna(0, inplace=True)
dfNext['vr'].fillna(100, inplace=True)
dfNext['vr_6_sma'].fillna(100, inplace=True)
#dfNext.loc[:, dfNext.isna().any()]
print(dfNext.columns[dfNext.isna().any()].tolist())
#print(dfNext[dfNext.isnull().any(axis=1)])

print(dfNext.dtypes)
dfNext.head()

[]
dealdate         object
dealtime         object
open            float64
high            float64
low             float64
close           float64
volume          float64
dealdatetime     object
dealdatei       float64
dealtimei       float64
testtend        float64
lastclose       float64
close_12_ema    float64
close_26_ema    float64
macd            float64
macds           float64
macdh           float64
close_-1_s      float64
close_-1_d      float64
rs_6            float64
rsi_6           float64
rs_12           float64
rsi_12          float64
rs_14           float64
rsi_14          float64
rsv_9           float64
kdjk_9          float64
kdjk            float64
kdjd_9          float64
kdjd            float64
                 ...   
wr_10           float64
wr_6            float64
close_10_sma    float64
close_50_sma    float64
dma             float64
high_delta      float64
um              float64
low_delta       float64
dm              float64
pdm             float64
pdm_14_ema   

,dealdate,dealtime,open,high,low,close,volume,dealdatetime,dealdatei,dealtimei,testtend,lastclose,close_12_ema,close_26_ema,macd,macds,macdh,close_-1_s,close_-1_d,rs_6,rsi_6,rs_12,rsi_12,rs_14,rsi_14,rsv_9,kdjk_9,kdjk,kdjd_9,kdjd,kdjj_9,kdjj,close_20_sma,close_20_mstd,boll,boll_ub,boll_lb,wr_10,wr_6,close_10_sma,close_50_sma,dma,high_delta,um,low_delta,dm,pdm,pdm_14_ema,pdm_14,tr,atr_14,pdi_14,pdi,mdm,mdm_14_ema,mdm_14,mdi_14,mdi,dx_14,dx,dx_6_ema,adx,adx_6_ema,adxr,change,vr,vr_6_sma
0,2018-06-29,08:45:00,10541.0,10542.0,10529.0,10531.0,4426.0,2018-06-29 08:45:00,20180629.0,84500.0,0.0,10523.0,10531.000000,10531.000000,0.000000,0.000000,0.000000,10531.0,0.0,0.000000,100.000000,0.000000,100.000000,0.000000,100.000000,15.384615,38.461538,38.461538,46.153846,46.153846,23.076923,23.076923,10531.000000,0.000000,10531.000000,10531.000000,10531.000000,84.615385,84.615385,10531.000000,10531.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,100.000000,100.000000
1,2018-06-29,08:46:00,10531.0,10532.0,10526.0,10531.0,2142.0,2018-06-29 08:46:00,20180629.0,84600.0,0.0,10523.0,10531.000000,10531.000000,0.000000,0.000000,0.000000,10531.0,0.0,0.000000,100.000000,0.000000,100.000000,0.000000,100.000000,31.250000,36.057692,36.057692,42.788462,42.788462,22.596154,22.596154,10531.000000,0.000000,10531.000000,10531.000000,10531.000000,68.750000,68.750000,10531.000000,10531.000000,0.0,-10.0,0.0,-3.0,3.0,0.0,0.000000,0.000000,6.0,6.000000,0.000000,0.000000,3.0,1.607143,1.607143,26.785714,26.785714,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.000000,100.000000,100.000000
2,2018-06-29,08:47:00,10530.0,10532.0,10530.0,10532.0,454.0,2018-06-29 08:47:00,20180629.0,84700.0,0.0,10523.0,10531.390300,10531.359290,0.031010,0.012709,0.036602,10531.0,1.0,0.000000,100.000000,0.000000,100.000000,0.000000,100.000000,37.500000,36.538462,36.538462,40.705128,40.705128,28.205128,28.205128,10531.333333,0.577350,10531.333333,10532.488034,10530.178633,62.500000,62.500000,10531.333333,10531.333333,0.0,0.0,0.0,4.0,0.0,0.0,0.000000,0.000000,2.0,3.925926,0.000000,0.000000,0.0,0.993209,0.993209,25.298715,25.298715,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.009501,142.390289,121.195145
3,2018-06-29,08:48:00,10531.0,10538.0,10529.0,10536.0,1204.0,2018-06-29 08:48:00,20180629.0,84800.0,0.0,10523.0,10532.845402,10532.656630,0.188772,0.072351,0.232843,10532.0,4.0,0.000000,100.000000,0.000000,100.000000,0.000000,100.000000,62.500000,45.192308,45.192308,42.200855,42.200855,51.175214,51.175214,10532.500000,2.380476,10532.500000,10537.260952,10527.739048,37.500000,37.500000,10532.500000,10532.500000,0.0,6.0,6.0,-1.0,1.0,6.0,1.835569,1.835569,9.0,5.744059,31.955964,31.955964,0.0,0.689358,0.689358,12.001240,12.001240,45.395800,45.395800,75.453158,75.453158,88.965181,88.965181,0.037980,254.808590,165.732960
4,2018-06-29,08:49:00,10537.0,10537.0,10527.0,10527.0,1522.0,2018-06-29 08:49:00,20180629.0,84900.0,0.0,10523.0,10531.257227,10531.344834,-0.087607,0.024767,-0.224748,10536.0,-9.0,0.447531,30.916844,0.500772,33.367609,0.508503,33.709132,6.250000,32.211538,32.211538,38.871083,38.871083,18.892450,18.892450,10531.400000,3.209361,10531.400000,10537.818723,10524.981277,93.750000,93.750000,10531.400000,10531.400000,0.0,-1.0,0.0,-2.0,2.0,0.0,1.356672,1.356672,10.0,6.929072,19.579420,19.579420,2.0,1.031303,1.031303,14.883706,14.883706,13.625328,13.625328,51.571462,51.571462,74.521436,74.521436,-0.085419,105.244890,150.610942


In [22]:
#dfNext['dealdate']=dfNext['dealdate'].astype('|S')
#dfNext['dealtime']=pd.to_string(dfNext['dealtime'])
#dfNext['dealdatetime']= pd.to_datetime(dfNext['dealdatetime'])

#dfNext['dealdate']=dfNext['dealdate'].replace("-","")
#dfNext['dealtime']=dfNext['dealtime'].replace(":","")

#dfNext['dealdate'] = pd.to_numeric(dfNext['dealdate'], downcast='float')
#dfNext['dealtime'] = pd.to_numeric(dfNext['dealtime'], downcast='float')

#dfNext['dealdatetimeL']=dfNext['dealdatetime'].strftime("%Y%m%d%H%M%S")
#dfNext[['dealdate','dealtime','dealdatetime','dealdatetimeL']].head()
print(dfNext[['dealdate','dealtime','dealdatetime']].dtypes)
dfNext[['dealdate','dealtime','dealdatetime']].head()

dealdate        object
dealtime        object
dealdatetime    object
dtype: object


,dealdate,dealtime,dealdatetime
0,2018-06-29,08:45:00,2018-06-29 08:45:00
1,2018-06-29,08:46:00,2018-06-29 08:46:00
2,2018-06-29,08:47:00,2018-06-29 08:47:00
3,2018-06-29,08:48:00,2018-06-29 08:48:00
4,2018-06-29,08:49:00,2018-06-29 08:49:00


好，接下來就從這裡用dfNext來進行LSTM的分析


0.   刪除
1. 分兩段df, training 與 verify, verify 取10 or 15天
2.   寫preprocess
  a. 把所有欄位值調整到0 to 1
  b. 利用deque將資料調為一段一段, 一段20根k棒
  c. 每20K棒產生一個趨勢，然後產生一堆所有如此的K棒
  d. randomize
  e. return ndarray 與 已知正確趨勢
3. 產生model
4. 最佳化
5. 比對verification set data
  



In [23]:
# training data and verification data separated

validation_dealdate=df2.tail(10)['dealdate'].min()

validation_dfNext = dfNext[(dfNext.dealdate >= validation_dealdate)]
dfNext = dfNext[(dfNext.dealdate < validation_dealdate)]

#validation_dfNext

# why testtend contains nan??!!
print(dfNext.columns[dfNext.isna().any()].tolist())
print(validation_dfNext.columns[validation_dfNext.isna().any()].tolist())
#dfNext.dtypes
#validation_dfNext[['dealdatetime','testtend']]
#print(validation_dfNext['testtend'].values.min())
#print(validation_dfNext['testtend'].values.max())

[]
[]


In [30]:

#dfNext
def preprocess_df(df): # df: input dataframe
    
    df["predictedTrend"]=0  # don't need this anymore.
    df=df.drop(['dealdate','dealtime','dealdatetime'], axis=1)
    for col in df.columns:  # go through all of the columns
        if col != "predictedTrend":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            #df.dropna(inplace=True)  # remove the nas created by pct_change
            df=df.replace([np.inf, -np.inf], np.nan) # replace all inf (infinite) with nan
            df.fillna(0, inplace=True)
            
            #print(df.columns[df.isna().any()].tolist())
            #print(col)
            
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.
        if col == "dealtimei":
            print(df[col].values)
            print(df[col].shape)
    #print(df.columns[df.isna().any()].tolist())
    print(df.head())
    print(df.shape)
    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in
    l=0
    for i in df.values:  # iterate over the values, each i become a sub array to be appended, this is to convert from dataframe to ndarray
        # i is df row, without index (time here)
        prev_days.append([n for n in i[:-1]])  # store all but the target
        '''
        if l<5:
            print([np.array(prev_days), i[-1]])
        '''
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!
        l=l+1
    print(prev_days) #very painful, deque cannot print anything but all
    #print(len(sequential_data))
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.
    
    X = []
    y = []
    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
    return np.array(X), y  # return X and y...and make X a numpy array!
validation_x, validation_y = preprocess_df(validation_dfNext)
print(len(validation_x[0]))
print(validation_x[0][0])
print(validation_y[0])

[-0.02129144  0.03506151  0.0349949  ...  0.01421807  0.01419161
  0.01416518]
(2986,)
          open      high       low  ...        vr  vr_6_sma  predictedTrend
3904 -0.014822 -0.015347 -0.015779  ... -0.070124 -0.032320               0
3905 -0.014822 -0.877216 -0.015779  ... -0.803865 -0.145854               0
3906 -0.831094 -1.164824 -0.302288  ...  0.080926 -0.272994               0
3907 -0.286988  0.559603  0.270755  ...  0.243743 -0.036943               0
3908  0.529560  0.272075  0.557237  ... -0.511044  0.039395               0

[5 rows x 65 columns]
(2986, 65)
deque([[-0.8198684914366093, 3.385972136292701, 0.2667613435978017, 3.435718808389395, 0.42428077339354264, -0.04763438938184526, 0.014674002979054683, 0.3941525639665095, -0.015075504503231857, 1.7516930425552633, 1.2036708958625784, 0.625254100897918, -0.2294197873965087, 0.1391358053174939, -0.7815453039506846, -2.4874372385062573, 0.9341871233485936, 1.5732839124833695, 3.251439888441244, 3.2855823149433876, 3.72018

In [0]:
#df2